# 本セクションの目次

# kafka / Spark Streaming / Avro
kafka / Spark Streaming / Avro はストリーミングシステムを作成する上での3種の神器です。
kafka と Spark Stramingについては前セクションで紹介しましたが、今回はそれに加えてAvroフォーマットについて学んでいきましょう

## Avro フォーマットとは？

もう一つがHadoopの生みの親であるDoug Cutting氏によりプロジェクト化されたAvro（アブロ）フォーマットです【URL】https://avro.apache.org。Avroフォーマットはおもにストリーミングでのやり取りで効力を発揮するフォーマットです余談ですが、同様のしくみとしてプロトコルバッファー（Protocol Buffers）は有名です。　【URL】https://developers.google.com/protocol-buffers/。 　元々AvroはHadoopの弱点であったJavaでしか読み書きできないという言語のポータビリティを解決するために生まれました言語のポータビリティーが低いということはそのままAvroファイルと連携する対向のシステムの利用言語まで縛ってしまう可能性があります。。 　Avroフォーマットの特徴は以下です。 ＠＠＠＠＠半行空き ・行指向フォーマット ・前方互換性と、後方互換性、完全互換を持ち複数のシステム間で速度の違う開発を行うことが可能 ・スキーマエボリューションを提供する ・Parquetに比べてJSONのようなリッチなフォーマットを表現可能

## スキーマファイルを保存する場所
スキーマファイルを保存する場所をスキーマレジストリと呼んだりします。
スキーマレジストリとは、共有してファイルを読み取れる場所である必要があります。

有名な企業としてConfulentが存在しますが、ただし今回はschemaのディレクトリをスキーマレジストリとします。


# Avroにおける前方互換、後方互換、完全互換

さまざまな言語で利用可能になった今、さらに注目されているAvroの特徴が開発スピードの違いを吸収することができるということです。一般にデータ基盤が相手をするシステムはスモールデータシステム含め社内のシステムすべてです。そのシステム群の開発のスピードを合わせようと思ったら組織が大きくなるにつれて調整のコストが増大し調整自体が不可能に近くなります。 　そこで、後方互換性や前方互換性というしくみが活躍します。後方互換性とは、新しい製品が、古い製品を扱えることを指します。前方互換性とは、古い製品が新しい製品を扱えることを指します。たとえば、Excel2010がExcel2003を扱えるようにすることを後方互換。Excel2003がExcel2010を扱えるようにすることを前方互換。ということを指します。 　後方互換や前方互換の機能を利用することによって、一方のシステムへ変更があったときでも、ほかのシステムの稼働を維持しつつ自システムの変更を行うことができるのです。このようなしくみを提供することをスキーマエボリューション（schema evolution）といいますParquetフォーマットはスキーマエボリューションの機能を有していません。なぜならば一方のシステムで変更を加えた場合、もう一方のシステムにも同時に変更を加えないとならないからです。。 　また、Avroフォーマットのもう一つの大きな特徴は、リッチなスキーマを表現できるという点にあります。キーバリューで表現できるmapやenumも表現可能でフォーマットの機能としてバリデーションも行ってくれます。 　Avroフォーマットにおけるスキーマ定義は以下のような形をしています。この定義に従ってデータ部分をシリアライズしたり、デシリアライズすることでデータを操作していきます。

# KafkaとAvroを連携してみよう
1. python(send avro with schema_ver1.avsc) -> kafka <- spark streaming(read avro with schema_ver1.avsc) -> memory
2. python(send avro with schema_ver1.avsc) -> kafka <- spark streaming(read avro with schema_ver1.avsc) -> parquet


1.の場合は今回はスキーマファイル(schema_ver1.avsc)を使ってシリアライズを実行しkafkaに転送します。  
kafkaに到着したデータはSpark Streamingによって弟子リアライズされよみだされ、コンソールに出力されます  

2.の場合は最後の出力がparquetに変換され出力されています。

# スキーマファイルの確認(schema_ver1.avsc)

まず今回のレクチャーで利用するAvroのシリアライズ/デシリアライズ用のスキーマです。  
送信したデータ(name/action/sendtime)がkafkaのvalueへ格納されることになります。

```
{
  "namespace": "root",
  "type": "record",
  "name": "Device",
  "fields": [
    { "name": "id", "type": "string" },
    { "name": "type", "type": "string" },
    { "name": "sendtime", "type": "int" }
  ]
}

```

Avroでは、スキーマは、4つの要素で成り立っています。

name − The value of this field holds the name of the record.
レコードの名前(今回だとDeviceに関するレコードですよ。)

namespace − The value of this field holds the name of the namespace where the object is stored.
データの名前空間

type − The value of this attribute holds either the type of the document (record) or the datatype of the field in the schema.
ほぼrecordだと思います。

fields − This field holds a JSON array, which have the list of all of the fields in the schema, each having name and the type attributes.
カラムの定義を行う部分です

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

In [ ]:
from confluent_kafka import Producer
import avro.schema
import avro.io
import io
import random

# スキーマファイルを読み込んでおきます
avro_json_schema = open("/home/pyspark/pyspark_streaming/schema/schema_ver1.avsc", "r").read()
avro_json_schema_ver2 = open("/home/pyspark/pyspark_streaming/schema/schema_ver2.avsc", "r").read()

In [ ]:
# PySparkをKafkaと接続します(kafka <- spark streaming(read avro))
# ストリームの経路を作成行います。

# kafkaからデータを読み取る設定を行います。

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic1") \
  .load()

In [ ]:
# spark streaming(read avro) -> console

console_stream_check = df \
  .select(from_avro("value", avro_json_schema).alias("json_col")) \
  .writeStream \
  .trigger(processingTime="5 seconds") \
  .format("console") \
  .option("checkpointLocation", "/tmp/kafka/console_check/") \
  .start()

In [ ]:
# spark streaming(read avro) -> parquet
from pyspark.sql.avro.functions import from_avro, to_avro

kafka_parquet = df \
  .select(from_avro("value", avro_json_schema).alias("json_col")) \
  .select("json_col.*") \
  .writeStream \
  .format("parquet") \
  .option("path", "/tmp/avro_parquet/") \
  .outputMode("append") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/avro_parquet/") \
  .start()

In [ ]:
from pyspark.sql.functions import window, col
from pyspark.sql.types import *

schema = StructType([
    StructField('id', StringType(), True),
    StructField('type', StringType(), True),
    StructField('sendtime', StringType(), True),
])

from pyspark.sql.avro.functions import from_avro, to_avro
from pyspark.sql import functions as F

# spark streaming(read avro) -> avro
kafka_avro = df.repartition(1).select("value").alias('Device') \
  .writeStream \
  .format("avro") \
  .option("path", "/tmp/avro_file/") \
  .outputMode("append") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/avro_kafka/") \
  .start()

In [ ]:
memory = df \
  .select(from_avro("value", avro_json_schema).alias("json_col")) \
  .writeStream \
  .format("memory") \
  .queryName("aggregates8") \
  .start()

In [ ]:
kafka_avro.stop()

In [ ]:
# 送信するpyspark 

conf = {'bootstrap.servers': 'kafka:9092'}
producer = Producer(**conf)

# Kafka topic
topic = "pyspark-topic1"

# Path to user.avsc avro schema
schema_path = "/home/pyspark/pyspark_streaming/schema/schema_ver1.avsc"
schema = avro.schema.parse(open(schema_path).read())

for i in range(1):
    writer = avro.io.DatumWriter(schema)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    # データの送信
    writer.write({"id": "yuki",
                    "type": "login2",
                    "sendtime": "time"}, encoder)
    raw_bytes = bytes_writer.getvalue()
    producer.produce(topic, raw_bytes)
producer.flush()

In [ ]:
spark.sql("select json_col.id from aggregates8").show()
spark.sql("select json_col.sendtime from aggregates8").show()

In [ ]:
console_stream_check.stop()
kafka_parquet.stop()

# ファイルがスモール？
ファイルが多すぎる場合は、以下の2点を調整することによって調節が可能です。

- プロセシング時間の延長
- repartitonの付与



# Avroファイルの読み書き
今回はストリーミングですが、一応Sparkでの読み込みも少しだけおさらいしておきましょう

Avroで出力するビッグデータとしての別のメリットはスプリッタブルである（データを分割して複数のサーバーに分けて処理を行うことが可能）という点です。

Avroに似ているフォーマットとしてJsonがありますが、こちらはスプリッタブルではなく（厳密には圧縮形式によります）データを分割して複数サーバーで処理するということに不向きです。

そのため「Jsonでいいや〜」と思わずに是非Avroでの実装を考えてみてください。

In [ ]:
# Avro でのデータの読み込み
avro_df = spark.read.format("avro").option('avroSchema', avro_json_schema).load("/tmp/avro_file8/")
avro_df.printSchema()
avro_df.show()

In [ ]:
# Avroでのデータ書き込み
avro_df.select("name").write.format("avro").save("/tmp/avro_etl/")

In [ ]:
! ls -al /tmp/avro_etl/

In [ ]:
spark.read.parquet("/tmp/avro_parquet/").show()

# Avroで後方互換をやってみよう
Avroの特徴の一つである後方互換をやってみましょう。

今回の後方互換のシナリオとしては、以下を想像してみてください。  
各家庭に冷蔵庫が配置されている(IoT機器、パブリッシャー)。  
そのIoT機器からは絶え間なくデータが流れている(ストリーミング)  
ストリーミングデータの受け口はKafkaを利用し、Spark Streamingでデータを読み込んでいる(コンシューマー)。

今回、IoTのソフトウェアがアップデートし温度(temp)も取得することが可能になったのでスキーマのアップデートを行いたい。  
しかし、各家庭に配置されている冷蔵庫のソフトウェアのアップデートを行うことは不可能である。

こんな時に活躍するのがAvroの後方互換です。

今回の後方互換はコンシューマー(Spark Streaming)が古いスキーマバージョンでシリアライズされたデータを新しいスキーマバージョンで読み取ることができるようにしたものです。  
この機能を使うことによって、システム側は早々にスキーマバージョンをアップデートし、ゆっくりと各家庭に配置されたソフトウェアが更新されるのを待てば良いことになります。

後方互換を行うために、shcema_ver2.avscというファイルを利用します。

```
{
  "namespace": "root",
  "type": "record",
  "name": "Device",
  "fields": [
    { "name": "id", "type": "string" },
    { "name": "type", "type": "string" },
    { "name": "sendtime", "type": "int" },
    { "name": "temp", "type": "string", "default": "1" }
  ]
}

```

古いスキーマバージョンからの送信であれば1にするという命令を後方互換として設定します。

In [ ]:
# Path to user.avsc avro schema
schema_path_ver1 = "/home/pyspark/pyspark_streaming/schema/schema_ver1.avsc"
schema_ver1 = avro.schema.parse(open(schema_path_ver1).read())


schema_path_ver2 = "/home/pyspark/pyspark_streaming/schema/schema_ver2.avsc"
schema_ver2 = avro.schema.parse(open(schema_path_ver2).read())

In [ ]:
# 送信するpyspark 
# ただし古いバージョンで送ります

conf = {'bootstrap.servers': 'kafka:9092'}
producer = Producer(**conf)

# Kafka topic
topic = "pyspark-topic1"

# Path to user.avsc avro schema
schema_path = "/home/pyspark/pyspark_streaming/schema/schema_ver1.avsc"
schema = avro.schema.parse(open(schema_path).read())

for i in range(1):
    writer = avro.io.DatumWriter(schema)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    # データの送信
    writer.write({"id": "yuki_schemaver1",
                    "type": "login2",
                    "sendtime": random.randint(0, 10)}, encoder)
    raw_bytes = bytes_writer.getvalue()
    producer.produce(topic, raw_bytes)
producer.flush()

In [ ]:
# 送信するpyspark 
# スキーマバージョン2で送信します

conf = {'bootstrap.servers': 'kafka:9092'}
producer = Producer(**conf)

# Kafka topic
topic = "pyspark-topic1"

# Path to user.avsc avro schema
schema_path_ver2 = "/home/pyspark/pyspark_streaming/schema/schema_ver2.avsc"
schema_ver2 = avro.schema.parse(open(schema_path_ver2).read())

for i in range(1):
    writer = avro.io.DatumWriter(schema_ver2)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    # データの送信
    writer.write({
                    "id": "yuki_schemaver2",
                    "type": "login2",
                    "sendtime": random.randint(0, 10),
                    "temp": "1"
                }, encoder)
    raw_bytes = bytes_writer.getvalue()
    producer.produce(topic, raw_bytes)
producer.flush()

In [ ]:
# spark streaming(read avro) -> console

memory_stream_check = df \
  .select(from_avro("value", avro_json_schema_ver2).alias("json_col")) \
  .writeStream \
  .trigger(processingTime="5 seconds") \
  .format("memory") \
  .queryName("check_kafka8") \
  .option("checkpointLocation", "/tmp/kafka/backword_check/") \
  .start()


In [ ]:
spark.sql("select * from check_kafka8").show()

In [ ]:
kafka_parquet.stop()
memory_stream_check.stop()

In [ ]:
spark.stop()